# 사전설정

In [2]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.map import FeatureGroup

In [ ]:
def filter_by_nan_ratio(group):
    nan_ratio = group['outlier'].isnull().sum() / len(group)
    return nan_ratio <= 0.8
def filter_by_outlier_ration(group):
    outlier = group['outlier']

route_station = pd.read_csv("완성데이터.csv", encoding="cp949")
filtered_route_station = route_station.groupby('노선_ID').filter(filter_by_nan_ratio)
filtered_route_station.to_csv("완성데이터(Nan 제외).csv", encoding="cp949")

In [6]:
#route_station = pd.read_csv("완성데이터(이상치 없음).csv", encoding="cp949")
route_station = pd.read_csv("완성데이터.csv", encoding="cp949")

target_data = filtered_route_station
bus_station = target_data.drop_duplicates(subset=["정류장_ID"], keep="first").copy()
bus_route = target_data.drop_duplicates(subset=['정류장_ID', '노선_ID'], keep="first").copy()

# 완성본

In [ ]:
grouped_routes = bus_route.groupby("노선_ID")
layer_control_data = {}
map=folium.Map(location=[37.566345, 126.977893],zoom_strat=6)

grouped_stations = route_station.groupby("노선_ID")


for route_id, route_group in grouped_routes:
    valid_points_line = route_group[route_group['위도'].notna() & route_group['경도'].notna()]

    if not valid_points_line.empty and len(valid_points_line) > 1:
        line_coords = valid_points_line[['위도', '경도']].values.tolist()
        route_name = route_group['노선명'].iloc[0]

        if pd.isna(route_name):
            route_name = str(route_id)
            PolyLine_popup = f"노선명 None, 노선_ID {str(route_id)}"
        else:
            PolyLine_popup = f"노선명 {route_name}, 노선_ID {str(route_id)}"

        feature_group = FeatureGroup(name=route_name, show=False)
        folium.PolyLine(line_coords, color="blue", weight=2.5, opacity=0.8, popup=PolyLine_popup).add_to(feature_group)

        if route_id in grouped_stations.groups:
            stations_of_route = grouped_stations.get_group(route_id)
            for index, station in stations_of_route.iterrows():
                stop_name = station['정류장명']
                latitude = station['위도']
                longitude = station['경도']
                outlier = station['outlier']
                if outlier == 0:
                    explain = '이상치 없음'
                elif outlier == 1:
                    explain = '승차 이상치'
                elif outlier == 2:
                    explain = '하차 이상치'
                else:
                    explain = 'Nan'

                if pd.notna(latitude) and pd.notna(longitude):
                    content = f"""
                        <div>
                            <h3>{stop_name} 역</h3>
                            <p>
                                이상치 상태 : {explain} </br>
                                [ 위도 : {latitude}, </br>
                                경도 : {longitude} ]
                            </p>
                        </div>
                    """
                    iframe = folium.IFrame(content, width=200, height=150)
                    popup = folium.Popup(iframe, max_width=2650)
                    #marker = folium.Marker(location=[latitude, longitude], popup=popup)
                    #marker = folium.CircleMarker(location=[latitude, longitude], radius=5, color='rightblue', fill=True, fill_color='green', fill_opacity=0.6, popup=popup)
                    try:
                        if outlier==0.0:
                            marker = folium.CircleMarker(location=[latitude, longitude], radius=5, color='rightblue', fill=True, fill_color='blue', fill_opacity=0.6, popup=popup)
                        elif outlier==1.0:
                            marker = folium.CircleMarker(location=[latitude, longitude], radius=10, color='rightred', fill=True, fill_color='red', fill_opacity=0.6, popup=popup)
                        elif outlier==2.0:
                            marker = folium.CircleMarker(location=[latitude, longitude], radius=10, color='rightgreen', fill=True, fill_color='green', fill_opacity=0.6, popup=popup)
                        else:
                            marker = folium.CircleMarker(location=[latitude, longitude], radius=5, color='rightgray', fill=True, fill_color='rightgray', fill_opacity=0.7, popup=popup)
                    except:
                        continue

                    marker.add_to(feature_group)

        feature_group.add_to(map)
        layer_control_data[route_name] = feature_group

    elif valid_points_line.empty:
        route_name = route_group['노선명'].iloc[0]
        if pd.isna(route_name):
            print(f"노선 ID {route_id}의 유효한 폴리라인 좌표가 없음")
        else:
            print(f"노선명 '{route_name}'의 유효한 폴리라인 좌표가 없음")

folium.LayerControl().add_to(map)
map.save("Bus_Routes_and_Stops.html")

---

In [24]:
import pandas as pd

bus_outliler = pd.read_csv("busoutlier.csv", encoding = "utf-8")
route_station = pd.read_csv("완성데이터(이상치 없음).csv", encoding="cp949")

route_station = pd.merge(route_station, bus_outliler[['노선_ID', '버스정류장ID', 'outlier']],
                        right_on=['노선_ID', '버스정류장ID'], left_on=['노선_ID', '정류장_ID'],
                        how = 'left')
route_station.drop("버스정류장ID", axis=1, inplace=True)

route_station.to_csv("완성데이터.csv", encoding="cp949", index=False)

In [ ]:
위도=bus_route['위도']
경도=bus_route['경도']
정류장명=bus_route['정류장명']
#도시명=bus_route['도시명']

mc = MarkerCluster()
map=folium.Map(location=[37.566345, 126.977893],zoom_strat=6)

for i in range(len((위도))):
    try:
        content=f"""
            <div>
                <h3>{정류장명.iloc[i]} 역</h3>
                <p>
                    [ 위도 : {위도.iloc[i]}, </br>
                    경도 : {경도.iloc[i]} ]
                </p>
            </div>
        """
        iframe=folium.IFrame(content, width=200, height=150)
        popup=folium.Popup(iframe, max_width=2650)

        marker=folium.Marker(location=[위도.iloc[i], 경도.iloc[i]], popup=popup)
        mc.add_child(marker)
    except:
        continue

map.add_child(mc)

In [ ]:
grouped = bus_route.groupby("노선_ID")
layer_control_data = {}

for route_id, group in grouped:
    valid_points = group[group['위도'].notna() & group['경도'].notna()]
    if not valid_points.empty and len(valid_points) > 1:
        line_coords = valid_points[['위도','경도']].values.tolist()

        route_name = group['노선명'].iloc[0]
        
        if pd.isna(route_name):
            route_name = str(route_id)

        feature_group = FeatureGroup(name = route_name, show=False)
        folium.PolyLine(line_coords, color="blue", weight=2.5, opacity=0.8).add_to(feature_group)
        feature_group.add_to(map)
        layer_control_data[route_name] = feature_group
    elif valid_points.empty:
        route_name = group['노선명'].iloc[0]
        if pd.isna(route_name):
            print(f"노선 ID {route_id} 자료 없음")
        else:
            print(f"노선명 '{route_name}'의 유효한 좌표점이 1개 이하")

folium.LayerControl().add_to(map)
map.save("Without How much people.html")

노선명 '마포순환열차버스'의 유효한 좌표점이 1개 이하


In [ ]:
위도=bus_route['위도']
경도=bus_route['경도']
정류장명=bus_route['정류장명']
#도시명=bus_route['도시명']

mc = MarkerCluster()
map=folium.Map(location=[37.566345, 126.977893],zoom_strat=6)

grouped = bus_route.groupby("노선_ID")
layer_control_data = {}

for route_id, group in grouped:
    valid_points = group[group['위도'].notna() & group['경도'].notna()]
    if not valid_points.empty and len(valid_points) > 1:
        line_coords = valid_points[['위도','경도']].values.tolist()
        route_name = group['노선명'].iloc[0]

        if pd.isna(route_name):
            route_name = str(route_id)

        feature_group = FeatureGroup(name = route_name, show=False)
        folium.PolyLine(line_coords, color="blue", weight=2.5, opacity=0.8).add_to(feature_group)

        # 각 좌표에 점을 추가하는 코드
        for lat, lon in line_coords:
            try:
                content=f"""
                    <div>
                        <h3>{정류장명.iloc[i]} 역</h3>
                        <p>
                            [ 위도 : {lat}, </br>
                            경도 : {lon} ]
                        </p>
                    </div>
                """
                iframe=folium.IFrame(content, width=200, height=150)
                popup=folium.Popup(iframe, max_width=2650)
                marker=folium.Marker(location=[lat, lon], popup=popup)
                
                marker.add_to(feature_group)
            except:
                continue
            #folium.CircleMarker(location=[lat, lon], radius=3, color="red", fill=True, fill_color="red", fill_opacity=0.7).add_to(feature_group)

        feature_group.add_to(map)
        layer_control_data[route_name] = feature_group
    elif valid_points.empty:
        route_name = group['노선명'].iloc[0]
        if pd.isna(route_name):
            print(f"노선 ID {route_id} 자료 없음")
        else:
            print(f"노선명 '{route_name}'의 유효한 좌표점이 1개 이하")

folium.LayerControl().add_to(map)
map.save("Without outliers.html")

노선명 '마포순환열차버스'의 유효한 좌표점이 1개 이하
